# Shivam Bomble
3MSAIM 2448510 NLP Lab 6

In [1]:
import numpy as np
import pandas as pd
import re
import random
import nltk
import tensorflow as tf
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# 📌 Sample Football Data
data = {
    "intent": [
        "greeting", "greeting", "greeting",
        "match_query", "match_query", "match_query",
        "player_info", "player_info", "player_info",
        "team_info", "team_info", "team_info",
        "goodbye", "goodbye"
    ],
    "text": [
        "Hello", "Hi", "Hey",
        "What was the latest football match?", "Who won the last football match?", "Give me today's match details.",
        "Tell me about Messi.", "Who is Cristiano Ronaldo?", "Give me details about Neymar.",
        "Tell me about Manchester United.", "What is Real Madrid's history?", "Which league does Bayern Munich play in?",
        "Bye", "See you later"
    ],
    "response": [
        "Hello! How can I help you with football today?", "Hi there! Ask me anything about football.", "Hey! Ready to talk football?",
        "The last match was between Team A and Team B. Team A won 2-1.", "The latest match was exciting! Team X secured the victory.", "Today's matches include Chelsea vs. Liverpool.",
        "Lionel Messi is an Argentine footballer who plays for Inter Miami.", "Cristiano Ronaldo is a Portuguese footballer known for playing at Manchester United and Real Madrid.", "Neymar is a Brazilian footballer currently playing for Al-Hilal.",
        "Manchester United is an English club based in Old Trafford.", "Real Madrid is a Spanish club with 14 UCL titles.", "Bayern Munich plays in the Bundesliga.",
        "Goodbye! Enjoy football.", "See you later! Have a great day!"
    ]
}

df = pd.DataFrame(data)

In [7]:
# 📌 Step 1: Preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)  # Remove special characters
    return text

df["text"] = df["text"].apply(preprocess_text)


Converts input to lowercase and removes special characters.

In [8]:
# 📌 Step 2: Intent Recognition using TF-IDF + Logistic Regression
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["text"])
y = df["intent"]

classifier = LogisticRegression()
classifier.fit(X, y)

LogisticRegression()

Uses TF-IDF vectorizer to convert text into features.
A Logistic Regression classifier is trained to predict intents.

In [12]:
# 📌 Step 3: Response Generation using LSTM
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["response"])
vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df["response"])
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Create LSTM model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dense(64, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert responses to labels
response_labels = np.array([i for i in range(len(df["response"]))])

# Train LSTM Model
model.fit(padded_sequences, response_labels, epochs=500, verbose=1)

Epoch 1/500
1/1 [==============================] - 13s 13s/step - loss: 4.4896 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 28ms/step - loss: 4.4831 - accuracy: 0.2143
Epoch 3/500
1/1 [==============================] - 0s 33ms/step - loss: 4.4771 - accuracy: 0.2857
Epoch 4/500
1/1 [==============================] - 0s 37ms/step - loss: 4.4707 - accuracy: 0.2857
Epoch 5/500
1/1 [==============================] - 0s 49ms/step - loss: 4.4634 - accuracy: 0.2143
Epoch 6/500
1/1 [==============================] - 0s 46ms/step - loss: 4.4548 - accuracy: 0.2143
Epoch 7/500
1/1 [==============================] - 0s 39ms/step - loss: 4.4445 - accuracy: 0.2143
Epoch 8/500
1/1 [==============================] - 0s 41ms/step - loss: 4.4323 - accuracy: 0.2143
Epoch 9/500
1/1 [==============================] - 0s 44ms/step - loss: 4.4174 - accuracy: 0.2143
Epoch 10/500
1/1 [==============================] - 0s 39ms/step - loss: 4.3991 - accuracy: 0.2143
Epoch 11/500
1/

Tokenizer converts responses into sequences.
LSTM model learns patterns in responses.
The model is trained to generate responses.

In [13]:
# 📌 Step 4: Chatbot Interaction
def chatbot_response(user_input):
    user_input = preprocess_text(user_input)
    
    # Step 1: Intent Recognition
    X_input = vectorizer.transform([user_input])
    predicted_intent = classifier.predict(X_input)[0]
    
    # Step 2: Generate Response
    filtered_responses = df[df["intent"] == predicted_intent]["response"].tolist()
    
    if filtered_responses:
        return random.choice(filtered_responses)
    else:
        return "Sorry, I don't understand that. Can you rephrase?"

The chatbot predicts intent using TF-IDF + Logistic Regression.
It retrieves a relevant response from predefined answers.

In [14]:
# 📌 Step 5: Running the Chatbot
print("⚽ Football Chatbot is ready! Type 'exit' to stop.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye! Have a great day! ⚽")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)


⚽ Football Chatbot is ready! Type 'exit' to stop.
Chatbot: Hello! How can I help you with football today?
Chatbot: Lionel Messi is an Argentine footballer who plays for Inter Miami.
Chatbot: Goodbye! Have a great day! ⚽
